# Feature Selection Techniques

Bu notebook içerisinde bazı Feature Selection (Özellik Seçimi) tekniklerine göz atacağız. Uygulanabilecek birçok farklı özellik tekniği olduğunu ve bu notebook içerisinde değerlendirilenler ile sınırlı kalmayacağını belirtmekte fayda var.

Feature Selection, kısaca bahsetmek gerekirse; veri seti içerisinde hedef değişken tahmini için kullanılan özellikleri, varyansı minimize edecek ve model performansını maksimize edecek şekilde ayıklamak için yapılan bir işlemdir. 

Örneğin veri seti içerisinde 5 adet özellik olduğunu varsayalım. Bu özelliklerin bazılarının korelasyonu yüksek olabilir ve bazılarının model içerisinde bulunmaması modelin genelleştirme gücünü yükseltiyor olabilir. Bu durumda sadece modele en faydalı olacak değişkenleri seçerek daha verimli bir model kurabiliriz.

Hazırlayan: Mert Yasin Bozkır

- https://bento.me/mybozkir

## Kütüphaneler

In [13]:
# Bu notebook'ta kullanacağımız mlxtend kütüphanesini yükleyelim.

%pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Kütüphaneler

#Genel Kütüphaneler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Model Selection
from sklearn.model_selection import train_test_split

# Feature Selection
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Regression Models
from sklearn.linear_model import LogisticRegression

# Datasets
from sklearn.datasets import load_iris

#Warnings
from warnings import filterwarnings
filterwarnings('ignore', category = DeprecationWarning)
filterwarnings('ignore', category = FutureWarning)


## Exhaustive Feature Selector (EFS)

Exhaustive Feature Selector, adındaki Exhaustive teriminin hakkını verecek şekilde veri seti içerisindeki tüm değişken kombinasyonlarını deneyerek en iyi sonucu veren özellik kümesini bize sunar.

Önce her değişkeni tek tek deneyerek arkasından ikili, üçlü ve (n adet değişken varsa) n'li kombinasyona kadar tüm alt kümeleri dener.

Bu işlem veri seti üzerine n değişkenin kombinasyonu kadar model kuracağı için çok fazla değişken sayısı olan veri setlerinde oldukça uzun süre alabileceği unutulmamalıdır. Bu nedenle bu tarz teknikleri uygulamadan önce veri setinde farklı değişken çıkarma veya seçme işlemleri uygulayarak bu noktaya gelmek daha mantıklı olacaktır.

İlk olarak kullanacağımız veri seti olan Iris veri setini yükleyelim.

Bu veri setini daha önce read_csv metodu ile yüklemiştik, şimdi bir farklılık yaparak Sci-kit'in datasets kütüphanesinden bu veri setini alalım.

Kütüphaneyi yukarıdaki Libraries bölümünde görebilirsiniz.

In [15]:
# Veri setini tutacağımız değişkene aktaralım.
iris = load_iris()

# Feature ve Target Values için set ayrımını yapalım.
X = iris.data     # data görevi görenleri bu şekilde çekebilirsiniz.
y = iris.target   # target values için ise bu şekilde alabilirsiniz.

Iris veri setini hatırlıyorsanız 4 adet özellik kolonumuz vardı.

Burada da aşağıdaki yapıda 4 kolon için ilk 10 gözlemi görebiliriz. Buradaki fark, bu yapının DataFrame olmaması.

İstersek bu yapıyı DataFrame'e aktarıp kolon isimlerini de verebiliriz fakat şu aşamada bunu yapmaya gerek yok.

In [16]:
X[:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [17]:
# İlk 10 adet sınıf değeri.
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [18]:
# İlk olarak train/test split işlemini yapalım.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

Şimdi Exhaustive Feature Selector kullanabiliriz. Bu sayade hangi değişkenlerin varlığı en iyi model sonucu için gerekiyor bunu göreceğiz.

Farklı modeller üzerinde denemeler yapmak mümkün olsa da şimdilik tek bir model üzerinden ilerleyelim.

EFS()'i kurarken ilk olarak kullanacağımız modeli veriyoruz. Bir döngü veya pipe ile farklı modelleri tek seferde deneyebilirsiniz, uzun süreceğiniz hatırlatırım.

min_features ve max_features en az ve en fazla kaç değişken kullanarak arama yapmak istediğinizi belirtir.

Şu anda sınıflama modeli kurduğumuz için scoring parametresine 'accuracy' değerini veriyoruz.

print_progress parametresi ile modele ilişkin ilerlemeyi print ettiriyoruz.

cv ise bildiğimiz Cross Validation, 5 katlamalı bir CV yaptıralım.

In [19]:
# Denemek istediğimiz modeli oluşturalım.
logreg = LogisticRegression(max_iter = 10000, penalty = None)

# EFS'i kuralım.
efs = EFS(logreg, min_features = 1, max_features = 4, scoring = 'accuracy', print_progress = True, cv = 5)

# EFS fit.
efs.fit(X_train, y_train)

# Sonuçları yazdıralım.
print('Best accuracy score: %.2f' % efs.best_score_)
print('Best subset (indices):', efs.best_idx_)
print('Best subset (names):', efs.best_feature_names_)

Features: 15/15

Best accuracy score: 0.97
Best subset (indices): (1, 2, 3)
Best subset (names): ('1', '2', '3')


Görebileceğimiz üzere en iyi subset (yani altküme) (1, 2, 3) index kümesi olarak belirlendi.

Eğer istersek denenmiş olan tüm altkümelerin performans skorlarına bakabiliriz.

In [20]:
efs.subsets_

{0: {'feature_idx': (0,),
  'cv_scores': array([0.83333333, 0.75      , 0.66666667, 0.79166667, 0.79166667]),
  'avg_score': 0.7666666666666666,
  'feature_names': ('0',)},
 1: {'feature_idx': (1,),
  'cv_scores': array([0.58333333, 0.54166667, 0.54166667, 0.45833333, 0.45833333]),
  'avg_score': 0.5166666666666667,
  'feature_names': ('1',)},
 2: {'feature_idx': (2,),
  'cv_scores': array([0.91666667, 1.        , 0.95833333, 1.        , 0.91666667]),
  'avg_score': 0.9583333333333334,
  'feature_names': ('2',)},
 3: {'feature_idx': (3,),
  'cv_scores': array([0.95833333, 0.95833333, 1.        , 0.91666667, 1.        ]),
  'avg_score': 0.9666666666666668,
  'feature_names': ('3',)},
 4: {'feature_idx': (0, 1),
  'cv_scores': array([0.875     , 0.91666667, 0.75      , 0.79166667, 0.875     ]),
  'avg_score': 0.8416666666666666,
  'feature_names': ('0', '1')},
 5: {'feature_idx': (0, 2),
  'cv_scores': array([0.91666667, 0.95833333, 0.95833333, 0.95833333, 0.95833333]),
  'avg_score': 0.

Yukarıda toplamda 15 kümenin denendiğini görebilirsiniz. Her küme numarasının karşısında dahil edilen özelliklerin indexleri, CV skorları, ortalama CV skoarları ve özelliklerin isimleri yazıyor.

get_metric_dict methodunu kullanarak daha detaylı ve temiz bir görünümle performansları görüntüleyebiliriz.

In [21]:
efs_scores = pd.DataFrame(efs.get_metric_dict()).T

efs_scores = efs_scores.sort_values(by = 'avg_score', ascending = False)

efs_scores

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
13,"(1, 2, 3)","[0.9583333333333334, 0.9583333333333334, 0.958...",0.975,"(1, 2, 3)",0.026236,0.020412,0.010206
3,"(3,)","[0.9583333333333334, 0.9583333333333334, 1.0, ...",0.966667,"(3,)",0.040076,0.03118,0.01559
14,"(0, 1, 2, 3)","[0.9166666666666666, 0.9583333333333334, 0.958...",0.966667,"(0, 1, 2, 3)",0.040076,0.03118,0.01559
7,"(1, 2)","[0.9166666666666666, 1.0, 0.9583333333333334, ...",0.966667,"(1, 2)",0.040076,0.03118,0.01559
2,"(2,)","[0.9166666666666666, 1.0, 0.9583333333333334, ...",0.958333,"(2,)",0.0479,0.037268,0.018634
9,"(2, 3)","[0.9166666666666666, 0.9583333333333334, 0.958...",0.958333,"(2, 3)",0.03387,0.026352,0.013176
6,"(0, 3)","[0.9166666666666666, 0.9583333333333334, 1.0, ...",0.958333,"(0, 3)",0.0479,0.037268,0.018634
8,"(1, 3)","[0.9583333333333334, 0.9583333333333334, 0.958...",0.958333,"(1, 3)",0.03387,0.026352,0.013176
5,"(0, 2)","[0.9166666666666666, 0.9583333333333334, 0.958...",0.95,"(0, 2)",0.021422,0.016667,0.008333
10,"(0, 1, 2)","[0.9166666666666666, 0.9583333333333334, 0.958...",0.95,"(0, 1, 2)",0.021422,0.016667,0.008333


Şimdi en iyi alt kümeyi kullanarak modelimizi tekrar kuralım ve test performansına bakalım.

In [22]:
logreg = LogisticRegression(max_iter = 10000, penalty = None)

X_train_subset = X_train[:, [1, 2, 3]]
X_test_subset = X_test[:, [1, 2, 3]]

logreg.fit(X_train_subset, y_train)

print(logreg.score(X_test_subset, y_test))

0.9666666666666667


## Sequential Feature Selector (SFS)

SFS, değişkenler arasında sıralı olarak deneme yapar. Forward ve Backward olmak üzere iki tipte çalışır.

Forward SFS, tek bir değişkenden başlayarak en iyi değişken performansını bulur. Sonra bulduğu değişkenin üzerine diğer değişkenleri tekrar ekleyerek ikili deneme yapar ve en iyi sonucu elde eden ikiliyi alır. Bu şekilde kaç değişkene kadar denemek istiyorsak o değişken sayısına kadar en iyi performansı verenleri alarak, ekleme yaparak devam eder.

Backward SFS ise Forward'ın tam tersi şekilde tüm değişkenleri alarak model kurar. Sonra tek bir değişken çıkartarak kombinasyonları dener, ta ki optimum değişkenleri bulana kadar çıkarma yaparak kombinasyonları denemeye devam eder.

Şimdi SFS instance'ını kurarak Forward ile denemeye başlayalım.

In [23]:
# SFS'i kuralım (Yine Logistic Regression ile yapalım).
sfs = SFS(logreg, k_features = 4, forward = True, floating = False, verbose = 2, scoring = 'accuracy', cv = 5)

# Fit işlemi.
sfs.fit(X_train, y_train)


[2024-10-06 13:28:11] Features: 1/4 -- score: 0.9666666666666668
[2024-10-06 13:28:11] Features: 2/4 -- score: 0.9583333333333334
[2024-10-06 13:28:11] Features: 3/4 -- score: 0.975
[2024-10-06 13:28:11] Features: 4/4 -- score: 0.9666666666666668

SequentialFeatureSelector(estimator=LogisticRegression(max_iter=10000,
                                                       penalty=None),
                          k_features=(4, 4), scoring='accuracy', verbose=2)

In [25]:
sfs.subsets_

{1: {'feature_idx': (3,),
  'cv_scores': array([0.95833333, 0.95833333, 1.        , 0.91666667, 1.        ]),
  'avg_score': 0.9666666666666668,
  'feature_names': ('3',)},
 2: {'feature_idx': (2, 3),
  'cv_scores': array([0.91666667, 0.95833333, 0.95833333, 0.95833333, 1.        ]),
  'avg_score': 0.9583333333333334,
  'feature_names': ('2', '3')},
 3: {'feature_idx': (1, 2, 3),
  'cv_scores': array([0.95833333, 0.95833333, 0.95833333, 1.        , 1.        ]),
  'avg_score': 0.975,
  'feature_names': ('1', '2', '3')},
 4: {'feature_idx': (0, 1, 2, 3),
  'cv_scores': array([0.91666667, 0.95833333, 0.95833333, 1.        , 1.        ]),
  'avg_score': 0.9666666666666668,
  'feature_names': ('0', '1', '2', '3')}}

In [26]:
sfs_forward_scores = pd.DataFrame.from_dict(sfs.get_metric_dict()).T

sfs_forward_scores = sfs_forward_scores.sort_values(by = 'avg_score', ascending = False)

sfs_forward_scores

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
3,"(1, 2, 3)","[0.9583333333333334, 0.9583333333333334, 0.958...",0.975,"(1, 2, 3)",0.026236,0.020412,0.010206
1,"(3,)","[0.9583333333333334, 0.9583333333333334, 1.0, ...",0.966667,"(3,)",0.040076,0.03118,0.01559
4,"(0, 1, 2, 3)","[0.9166666666666666, 0.9583333333333334, 0.958...",0.966667,"(0, 1, 2, 3)",0.040076,0.03118,0.01559
2,"(2, 3)","[0.9166666666666666, 0.9583333333333334, 0.958...",0.958333,"(2, 3)",0.03387,0.026352,0.013176


Görebileceğimiz üzere yine aynı değişkenleri verdi. Şimdi bir de Backward olarak deneyelim.

In [29]:
# SFS'i kuralım (Yine Logistic Regression ile yapalım). Bu sefer forward = False diyerek Backward deniyoruz.
sfs = SFS(logreg, k_features = 1, forward = False, floating = False, verbose = 2, scoring = 'accuracy', cv = 5)

# Fit işlemi.
sfs.fit(X_train, y_train)

# Skorları kaydedelim.
sfs_backward_scores = pd.DataFrame.from_dict(sfs.get_metric_dict()).T

sfs_backward_scores = sfs_backward_scores.sort_values(by = 'avg_score', ascending = False)

sfs_backward_scores


[2024-10-06 13:31:56] Features: 3/1 -- score: 0.975
[2024-10-06 13:31:56] Features: 2/1 -- score: 0.9666666666666666
[2024-10-06 13:31:56] Features: 1/1 -- score: 0.9583333333333334

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
3,"(1, 2, 3)","[0.9583333333333334, 0.9583333333333334, 0.958...",0.975,"(1, 2, 3)",0.026236,0.020412,0.010206
4,"(0, 1, 2, 3)","[0.9166666666666666, 0.9583333333333334, 0.958...",0.966667,"(0, 1, 2, 3)",0.040076,0.03118,0.01559
2,"(1, 2)","[0.9166666666666666, 1.0, 0.9583333333333334, ...",0.966667,"(1, 2)",0.040076,0.03118,0.01559
1,"(2,)","[0.9166666666666666, 1.0, 0.9583333333333334, ...",0.958333,"(2,)",0.0479,0.037268,0.018634


Şu ana kadar EFS ve SFS'i gördük.

Şimdi daha önce kullandığımız diabetesdata veri seti kullanarak tekrar selection işlemi yapalım. Bu veri setinde daha fazla değişken olacak, farkı görmek daha kolay olabilir.

In [31]:
# Veri setini yükleyelim.

df = pd.read_table('../../datasets/classification/diabetesdata.txt')

df

,Preg,Glucose,BP,SkinThick,Insul,BMI,DPF,Age,Diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [32]:
# X ve y ayrımını yapalım.
X = df.drop(columns = ['Diabetes'], axis = 1)
y = df['Diabetes']

In [33]:
# Train/test split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [34]:
# EFS - Logistic Regression
logreg = LogisticRegression(max_iter = 10000, penalty = None)

efs = EFS(logreg, min_features = 1, max_features = 8, scoring = 'accuracy', print_progress = True, cv = 5)

efs.fit(X_train, y_train)

# Skorları kaydedelim.
efs_scores = pd.DataFrame(efs.get_metric_dict()).T

efs_scores = efs_scores.sort_values(by = 'avg_score', ascending = False)

efs_scores

Features: 255/255

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
69,"(1, 5, 6)","[0.7886178861788617, 0.7886178861788617, 0.739...",0.786712,"(Glucose, BMI, DPF)",0.035905,0.027936,0.013968
239,"(1, 2, 3, 4, 5, 6)","[0.7642276422764228, 0.7886178861788617, 0.747...",0.783487,"(Glucose, BP, SkinThick, Insul, BMI, DPF)",0.042537,0.033095,0.016547
170,"(0, 1, 2, 5, 7)","[0.7967479674796748, 0.7723577235772358, 0.772...",0.783433,"(Preg, Glucose, BP, BMI, Age)",0.022832,0.017764,0.008882
211,"(1, 4, 5, 6, 7)","[0.7479674796747967, 0.8130081300813008, 0.764...",0.783407,"(Glucose, Insul, BMI, DPF, Age)",0.030493,0.023725,0.011862
181,"(0, 1, 5, 6, 7)","[0.7804878048780488, 0.7967479674796748, 0.772...",0.783407,"(Preg, Glucose, BMI, DPF, Age)",0.013687,0.010649,0.005324
...,...,...,...,...,...,...,...
87,"(3, 6, 7)","[0.6341463414634146, 0.6260162601626016, 0.650...",0.649833,"(SkinThick, DPF, Age)",0.028845,0.022442,0.011221
35,"(6, 7)","[0.6422764227642277, 0.6097560975609756, 0.666...",0.648181,"(DPF, Age)",0.03627,0.028219,0.014109
6,"(6,)","[0.6178861788617886, 0.6422764227642277, 0.658...",0.646595,"(DPF,)",0.020008,0.015567,0.007783
155,"(2, 4, 6, 7)","[0.6422764227642277, 0.6341463414634146, 0.650...",0.646595,"(BP, Insul, DPF, Age)",0.009725,0.007567,0.003783


Görebileceğimiz üzere bu sefer 255 farklı kombinasyon denendi. Ortalama skoru en iyi olan kombinasyon Glucose, BMI ve DPF olarak gözüküyor.

Şimdi bu kolonlar ile model kurarak test performansına bakalım.

In [40]:
X_train_subset = X_train.iloc[:, list(efs.best_idx_)] #En iyi indexleri bu şekilde de verebiliriz, elle yazmaktan daha iyi bir çözüm.
X_test_subset = X_test.iloc[:, list(efs.best_idx_)]

In [41]:
# Test performansını ölçüyoruz.
logreg = LogisticRegression(max_iter = 10000, penalty = None)

logreg.fit(X_train_subset, y_train)

print(logreg.score(X_test_subset, y_test))

0.7207792207792207


Seçilen indexler ile ortalama CV skorumuz 0.786712 çıkmışken test seti performansımız 0.720779 çıktı.

In [43]:
# SFS Forward
logreg = LogisticRegression(max_iter=1000,penalty=None)
sfs = SFS(logreg, 
           k_features=8, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=5)

sfs = sfs.fit(X_train, y_train)
pd.DataFrame.from_dict(sfs.get_metric_dict()).T.sort_values(by = 'avg_score', ascending = False)


[2024-10-06 13:42:41] Features: 1/8 -- score: 0.7541383446621351
[2024-10-06 13:42:41] Features: 2/8 -- score: 0.7769425563108091
[2024-10-06 13:42:42] Features: 3/8 -- score: 0.786711981873917
[2024-10-06 13:42:42] Features: 4/8 -- score: 0.7818605890976943
[2024-10-06 13:42:42] Features: 5/8 -- score: 0.783406637345062
[2024-10-06 13:42:42] Features: 6/8 -- score: 0.783406637345062
[2024-10-06 13:42:42] Features: 7/8 -- score: 0.7817806210848994
[2024-10-06 13:42:42] Features: 8/8 -- score: 0.7769159003065441

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
3,"(1, 5, 6)","[0.7886178861788617, 0.7886178861788617, 0.739...",0.786712,"(Glucose, BMI, DPF)",0.035905,0.027936,0.013968
5,"(1, 4, 5, 6, 7)","[0.7479674796747967, 0.8130081300813008, 0.764...",0.783407,"(Glucose, Insul, BMI, DPF, Age)",0.030493,0.023725,0.011862
6,"(1, 3, 4, 5, 6, 7)","[0.7479674796747967, 0.8130081300813008, 0.764...",0.783407,"(Glucose, SkinThick, Insul, BMI, DPF, Age)",0.030493,0.023725,0.011862
4,"(1, 4, 5, 6)","[0.7560975609756098, 0.7804878048780488, 0.739...",0.781861,"(Glucose, Insul, BMI, DPF)",0.04588,0.035697,0.017848
7,"(1, 2, 3, 4, 5, 6, 7)","[0.7479674796747967, 0.8130081300813008, 0.772...",0.781781,"(Glucose, BP, SkinThick, Insul, BMI, DPF, Age)",0.028076,0.021844,0.010922
2,"(1, 5)","[0.7642276422764228, 0.7804878048780488, 0.747...",0.776943,"(Glucose, BMI)",0.030743,0.023919,0.01196
8,"(0, 1, 2, 3, 4, 5, 6, 7)","[0.7642276422764228, 0.7804878048780488, 0.764...",0.776916,"(Preg, Glucose, BP, SkinThick, Insul, BMI, DPF...",0.01863,0.014495,0.007247
1,"(1,)","[0.7967479674796748, 0.7479674796747967, 0.707...",0.754138,"(Glucose,)",0.046896,0.036486,0.018243


Aynı indexleri elde ettik, tekrar model kurmayacağız.

Şimdi bir de Backward yapalım.

In [47]:
# SFS Backward
logreg = LogisticRegression(max_iter=1000,penalty=None)
sfs = SFS(logreg, 
           k_features=1, 
           forward=False, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=5)

sfs = sfs.fit(X_train, y_train)
scores = pd.DataFrame.from_dict(sfs.get_metric_dict()).T.sort_values(by = 'avg_score', ascending = False)
scores


[2024-10-06 13:45:36] Features: 7/1 -- score: 0.7817939490870318
[2024-10-06 13:45:36] Features: 6/1 -- score: 0.7818072770891644
[2024-10-06 13:45:36] Features: 5/1 -- score: 0.7834332933493269
[2024-10-06 13:45:37] Features: 4/1 -- score: 0.7801812608290019
[2024-10-06 13:45:37] Features: 3/1 -- score: 0.7785552445688392
[2024-10-06 13:45:37] Features: 2/1 -- score: 0.7769425563108091
[2024-10-06 13:45:37] Features: 1/1 -- score: 0.7541383446621351

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
5,"(0, 1, 2, 5, 7)","[0.7967479674796748, 0.7723577235772358, 0.772...",0.783433,"(Preg, Glucose, BP, BMI, Age)",0.022832,0.017764,0.008882
6,"(0, 1, 2, 4, 5, 7)","[0.7967479674796748, 0.7723577235772358, 0.772...",0.781807,"(Preg, Glucose, BP, Insul, BMI, Age)",0.025337,0.019713,0.009856
7,"(0, 1, 2, 4, 5, 6, 7)","[0.7642276422764228, 0.7723577235772358, 0.788...",0.781794,"(Preg, Glucose, BP, Insul, BMI, DPF, Age)",0.017316,0.013472,0.006736
4,"(0, 1, 2, 5)","[0.7967479674796748, 0.7642276422764228, 0.772...",0.780181,"(Preg, Glucose, BP, BMI)",0.026649,0.020734,0.010367
3,"(0, 1, 5)","[0.8048780487804879, 0.7560975609756098, 0.747...",0.778555,"(Preg, Glucose, BMI)",0.032794,0.025515,0.012757
2,"(1, 5)","[0.7642276422764228, 0.7804878048780488, 0.747...",0.776943,"(Glucose, BMI)",0.030743,0.023919,0.01196
8,"(0, 1, 2, 3, 4, 5, 6, 7)","[0.7642276422764228, 0.7804878048780488, 0.764...",0.776916,"(Preg, Glucose, BP, SkinThick, Insul, BMI, DPF...",0.01863,0.014495,0.007247
1,"(1,)","[0.7967479674796748, 0.7479674796747967, 0.707...",0.754138,"(Glucose,)",0.046896,0.036486,0.018243


Bu sefer farklı indexler elde ettik, bakalım bu indexler ile test skorumuz ne olacak.

In [56]:
X_train_subset = X_train.iloc[:, list(scores['feature_idx'].iloc[0])] 
X_test_subset = X_test.iloc[:, list(scores['feature_idx'].iloc[0])]

# Test performansını ölçüyoruz.
logreg = LogisticRegression(max_iter = 10000, penalty = None)

logreg.fit(X_train_subset, y_train)

print(logreg.score(X_test_subset, y_test))

0.7207792207792207
